In [2]:
setwd('/root/miniconda3/envs/Rdoc/share/homer')

In [3]:
subString <- function(strings, idx, sep = NA){

    strings = as.character(strings)
    if(is.na(sep)){
        res = as.character(lapply(strings, function(x) paste(strsplit(x, "")[[1]][idx], collapse = "")))
    } else{
        res = sapply(strsplit(strings, sep), function(x) x[idx])
    }
    return(res)
}

summaryHomer <- function(outFolder){

    homerFolder = paste0(outFolder, "/homerResults")
    xFiles = list.files(homerFolder, ".motif$")
    xFiles = xFiles[-grep("similar", xFiles)]
    xFiles = xFiles[-grep("RV", xFiles)]
    xFiles = xFiles[order(as.numeric(gsub("\\.", "", gsub("motif", "", xFiles))))]
    texts  = sapply(paste0(homerFolder, "/", xFiles), readLines)
    chunks = sapply(texts, function(x) strsplit(x[1], "[\t]"))

    motif = sapply(chunks, function(x) subString(x[1], 2, ">"))
    match = sapply(chunks, function(x) subString(subString(x[2], 2, "BestGuess:"),  1, "/"))
    score = sapply(chunks, function(x) rev(strsplit(x[2], "[()]")[[1]])[1])
    count = sapply(chunks, function(x) subString(x[6], 3, "[T:()]"))
    ratio = sapply(chunks, function(x) subString(x[6], 2, "[()]"))
    p_value = sapply(chunks, function(x) subString(x[6], 2, "P:"))

    xresT = data.frame(motif, 
                       match, 
                       score = as.numeric(score), 
                       count = as.numeric(count),
                       ratio_perc = as.numeric(gsub("%", "", ratio)), 
                       p_value = as.numeric(p_value)
                       )
    rownames(xresT) = gsub(".motif", "", basename(rownames(xresT)))
    return(xresT)
}

In [4]:
rela = summaryHomer('out')

In [5]:
IKBKB = summaryHomer('out3')

In [6]:
NFKBIA = summaryHomer('out4')

In [19]:
# write.csv(rela,file = 'RELA.csv')
# write.csv(IKBKB,file = 'IKBKB.csv')
# write.csv(NFKBIA,file = 'NFKBIA.csv')

In [9]:
intersect(rela$match,IKBKB$match)

[1] "NFYB"          "POL006.1_BREu"

In [10]:
intersect(NFKBIA$match,IKBKB$match)

[1] "ZNF354C"      "ZNF341"       "MEIS2(var.2)"

In [11]:
intersect(NFKBIA$match,rela$match)

[1] "RUNX2"           "PB0137.1_Irf3_2" "Ddit3::Cebpa"    "PH0037.1_Hdx"   
[5] "MYNN(Zf)"

In [12]:
intersect(intersect(NFKBIA$match,rela$match),IKBKB$match)

character(0)

In [14]:
library(ggvenn)

In [21]:
a <- list(`NFKBIA` = NFKBIA$match,
          `rela` = rela$match
          )
pdf('/import/code/homre.res/NFKBIA_RELA.pdf')
ggvenn(a, c("NFKBIA", "rela"))
dev.off()

png 
  2

In [22]:
a <- list(`rela` = rela$match,
          `IKBKB` = IKBKB$match
          )
pdf('/import/code/homre.res/IKBKB_RELA.pdf')
ggvenn(a, c("rela", "IKBKB"))
dev.off()

png 
  2

In [23]:
a <- list(`NFKBIA` = NFKBIA$match,
          `IKBKB` = IKBKB$match
          )
pdf('/import/code/homre.res/IKBKB_NFKBIA.pdf')
ggvenn(a, c("NFKBIA", "IKBKB"))
dev.off()

png 
  2